# Python Open GL Examples

## Import

In [1]:
from OpenGL.GL import *
from OpenGL.GLUT import *
from OpenGL.GLU import *
import threading
import time

## Hello triangle with Legacy OpenGL

In [2]:
class OpenGLApp:
    def __init__(self, background_col=(1., 1., 1., 1.0), screen_size=(800, 600)):
        self.background_col = background_col
        self.screen_size = screen_size
        self.running = False

    def reshape(self, w, h):
        self.screen_size = (w, h)
        glViewport(0, 0, w, h)
        glLoadIdentity()
        gluPerspective(45, w / h, 0.1, 50.)
        glMatrixMode(GL_MODELVIEW)
        glTranslatef(0, 0, -3)
    
    def handle_keys(self, key, x, y):
        if key == b'\x1b':
            self.close()

    def close(self):
        self.running = False
        glutLeaveMainLoop()
    
    def run(self):
        glutInit()
        glutInitDisplayMode(GLUT_DOUBLE | GLUT_RGB | GLUT_DEPTH)
        glutInitWindowSize(*self.screen_size)
        glutCreateWindow(b'OpenGL Window')
        glClearColor(*self.background_col)
        glutDisplayFunc(self.update)
        glutReshapeFunc(lambda w, h: self.reshape(w, h))
        glutKeyboardFunc(lambda k, x, y : self.reshape(k, x, y))
        glutWMCloseFunc(self.close)

        self.running = True
        while self.running:
            glutMainLoopEvent()
            time.sleep(0.01)
    
    def update(self):
        raise NotImplementedError

In [3]:
class LegacyOpenGLApp(OpenGLApp):
    def __init__(self, vertices, colors):
        super().__init__()
        self.vertices = vertices
        self.colors = colors

    def update(self):
        ''' Override the update to legacy open GL triangles. ''' 
        glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)

        glBegin(GL_TRIANGLES)
        for color, vertex in zip(self.colors, self.vertices):
            glColor3fv(color)
            glVertex3fv(vertex)
        glEnd()

        glutSwapBuffers()

In [4]:
vertices = [[-0.5, -0.5, 0], [0.5, -0.5, 0], [0, 0.5, 0]]
colors = [[1., 0., 0.], [0., 1., 0.], [0., 0., 1.]]

In [5]:
app = LegacyOpenGLApp(vertices, colors)
thread = threading.Thread(target=app.run)
thread.start()

## Disaplaying on .obj file

In [6]:
from pygments import highlight
from pygments.lexers import get_lexer_by_name
from pygments.formatters import HtmlFormatter
from IPython.display import HTML, display

In [7]:
def highlight_code(path, lexer_name='glsl', line_range=None):
    with open(path) as f:
        contents = f.read()

    if line_range is not None:
        start, end = line_range
        contents = '\n'.join(contents.split('\n')[start:end])
    
    lexer = get_lexer_by_name(lexer_name)
    formatter = HtmlFormatter(style='colorful')
    highlighted_code = highlight(contents, lexer, formatter)
    highlighted_code = f'<style>{formatter.get_style_defs(".highlight")}</style>{highlighted_code}'
    display(HTML(highlighted_code))

In [8]:
highlight_code('./Shaders/texture.frag')

In [15]:
highlight_code('./ObjFiles/Bulbasaur/Bulbasaur.obj', line_range=(0, 6))

In [17]:
highlight_code('./ObjFiles/Bulbasaur/Bulbasaur.obj', line_range=(369, 371))

In [18]:
highlight_code('./ObjFiles/Bulbasaur/Bulbasaur.obj', line_range=(485, 487))

## Diffuse and Specular light

## Adding movement